In [1]:
P = Primes()
ZR.<x> = PolynomialRing(ZZ)
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
C.<x> = PolynomialRing(CC)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m,x): 
    f=J(n,m,x)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m,x):
    return xjNewStep(n-1,m,x)
def xjNewCoefficient(n,m,x):
    if n>0:return (xjNew(n,m,x)-xjNew(n-1,m,x))/x^n
    if n==0:return 1
def xjNewTest(n,m,x):
    return D(xjNewStep(n-1,m,x))
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def prime_n(a):return P.unrank(a-1)

def primeFactors(n):
    answer=[]
    lf=list(n.factor())
    length=len(lf)
    for k in [0..length-1]:
        pair=lf[k]
        (base,exponent)=pair
        answer=answer+[[base]]
    return(flatten(answer))

def primeFactorsInList(lst):
    ld=len(lst)
    primelist=[]
    for k in [0..ld-1]:
        a=ZZ(lst[k])
        primelist=primelist+[primeFactors(a)]
    primelist=flatten(primelist)
    primelist=list(set(primelist))
    return primelist

def den(a):
    if a in QQ:
        if a in ZZ:return 1
        return a.denominator()
def listdens(lst):
    return list(den(a) for a in lst)
def polynomialdenominator(poly):return lcm(listdens(poly))

def polynomialnumerator(poly):
    return expand(poly*polynomialdenominator(poly))

def polynomialnumeratorP(p,poly):
    pd=polynomialdenominator(poly)
    od=ord(p,pd)
    return expand(poly*p^od)

def num(a):return a.numerator()
def prime(n):
    P=Primes()
    return P.unrank(n-1)
def inclusiveListPrimesTo(n):
    if is_prime(n):
        return list(primes(n+1))
    else:
        return list(primes(n))
def chunk(k,list):
    chnk=[]
    for n in [k..len(list)-1]:
        chnk=chnk+[list[n]]
    return chnk
def drop(m,lst): return list(Set(lst).difference({m}))
def sort(list):
    answr=[]
    chnk=list
    for n in [0..len(list)-1]:
        m = min(chnk)
        answr=answr+[m]
        chnk=drop(m,chnk)
    return answr
def ord(p,n):return QQ.valuation(p)(n)
def base_conversion(number, base_from, base_to):  # by Vishnu Namboothiri
# https://ask.sagemath.org/question/9657/how-to-output-decimal-numbers-in-different-radix/
    if(base_from==10):
        return stripQuotationMarks((number.str(base=base_to)))
    else:
        decimal = Integer(str(number), base=base_from)
        return (stripQuotationMarks(decimal.str(base=base_to)))
def lastFactorWithIntegralFactors(polynomial):
    return polynomialnumerator(lastfactor(polynomial))
def numericalFactorWithIntegralFactors(polynomial):
    nf=numericalfactor(polynomial);
    pd=polynomialdenominator(lastfactor(polynomial))
    return nf/pd


def primeDivisors(n):
    pd=[]
    n=n
    fn=factor(n)
    ln=len(fn)
    for k in [0..ln-1]:
        part = fn[k]
        (prime,exponent) = part
        pd = pd + [prime]
    return pd 

def fOverField2(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    return ppoly

###################### following two routines disagree ####################
def factorOverField(f,prime,power):
    answer=[]
    T = GF(prime^power,'t')
    G.<t>= GF(prime^power)
    U.<x> = T[]
    ppoly = U(f)
    expr=str(ppoly)
    var('x')
    fp=ppoly.factor()
    return fp

def factorOverField2(poly,prime,power,j):
    sd = leastSplittingDegree(poly,prime,bound)
    T=GF(prime^j)
    G.<t>= GF(prime^sd)
    U.<x> = T[]
    ppoly=U(poly)
    fpp=ppoly.factor()
    return fpp

############################

#######################################################
# splitting tests below fail at  e.g. f = x + 1 
############################################################

def splitsTF(f,prime,power):
    T = GF(prime^power)
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx > 1:return(False)
    return(True)

def leastSplittingDegree(f,prime,bound):
    for power in [1..bound]:
        tf=splitsTF(f,prime,power)
        if tf == True:return power
    return -1

#########################################################

def firstNprimes(n):
    P=Primes()
    first=[]
    for k in [0..n-1]:
        first=first+[P.unrank(k)]
    return first

def slopeExistsTF(pairlist):
    lenth=len(pairlist)
    if lenth<3:return False
    slopes=[]
    for k in [1..lenth-1]:
        pair1=pairlist[k-1]
        abscissa1=pair1[0]
        ordinate1=pair1[1]
        pair2=pairlist[k]
        abscissa2=pair2[0]
        ordinate2=pair2[1]
        dx=abscissa2-abscissa1
        if dx == 0:return False
        dy=ordinate2-ordinate1
        slope=dy/dx
        slopes=slopes+[slope]
    slopes=Set(slopes)
    if len(slopes) == 1:
        return [True,slopes[0]]
    if len(slopes) > 1:
        return False 
    
def zeroOfLinearMonomial(monomial,x):return x-monomial

def totalDegreeOverField(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    td=0
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        td=td+exponent
    return td

def logsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def discreteLogOverField(xm,prime,power):
    T = GF(prime^power,'t')
    t=T.multiplicative_generator()
    U.<x> = T[]
    if xm != 0:
        for i in [0..prime^power-1]:
            if t^i==xm:return i

def generatingZeroLogsOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        if mod(i,prime)>0:
                            data=data+[i]
    data=data.sort()
    return data



def generatingZeroLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        if mod(i,prime)>0:
                            data=data+[[i,exponent]]
    return data




def logsAndMultiplicitiesOverFieldTest(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data

def logsAndMultiplicitiesOverFieldNoPrint(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:data=data+[[i,exponent]]
    return data


def dropzeros(list):
    data=[]
    el=len(list)
    for j  in [0..el-1]:
        lj=list[j]
        if lj != 0:data=data+[lj]
    return data

def padListOnRightToLengthK(listo,k):
    length=len(listo)
    padded=listo
   
    if length<k:
        for j in [length..k-1]:
            padded=padded+[0]
           
        return padded
    return listo


def logsAndMultiplicitiesOverFieldDigits(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:
                i=[-Infinity]
                k=power
                i=padListOnRightToLengthK(i,k)
                data=data+[[i,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if b^i==xm:
                        i=i.digits(prime)
                        k=power
                        i=padListOnRightToLengthK(i,k)
                        data=data+[[i,exponent]]
    return data


def testCandidateZeroLogOverField(f,prime,power,candidate):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    a=t^candidate
    expr3=sage_eval(expr,locals={'x':a})
    if expr3 == 0:tf=True
    return tf

def digitsToInteger(digitlist,prime):
    lenth=len(digitlist)
    sum=0
    for k in [0..lenth-1]:
        sum=sum+prime^k*digitlist[k]
    return sum

def minimumIntegerRepresentedInList(lstlst,prime):
    answer=Infinity
    ln=len(lstlst)
    for k in [0..ln-1]:
        lst=lstlst[k]
        n=digitsToInteger(lst,prime)
        if n < answer:answer=n
    return answer

def lastElement(lst):
    ln=len(lst)
    return lst[ln-1]

def push(lst):
    if len(lst)<2:return lst
    answer=[lastElement(lst)]
    for k in [0..len(lst)-2]:
        answer=flatten([answer]+[lst[k]])
    return answer

def cyclicPermutationEquivalenceClass(lst):
    if lst==[]:return lst
    answer=[lst]
    lnth=len(lst)
    for k in [0..lnth-2]:
        ple=push(lastElement(answer))
        answer=answer+[ple]
    return answer

def bestRepresentativeOfCyclicEquivalenceClass(lst,prime):
    cpec=cyclicPermutationEquivalenceClass(lst)
    minm=minimumIntegerRepresentedInList(cpec,prime)
    lcpec=len(lst) # ... = len(cpec)
    for k in [0..lcpec-1]:
        ck=cpec[k]
        dti=digitsToInteger(ck,prime)
        if minm==dti:return ck
    return "error"

def directZeroLogOverField(f,prime,power):
    tf=False
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    answer=[]
    expr3=sage_eval(expr,locals={'x':0})
    if expr3 == 0:
        answer=[0]
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[i]
    return answer

def reduce(f,prime):
    poly=polynomialnumerator(D(f))
    
    ls=list(poly)
    
    def modulo(n):
        return ZZ(mod(n,prime))
    def listmod(list):
        return map(modulo,list)
    lmls = listmod(ls)
    sm=0
    for j in [0..len(lmls)-1]:
        sm=sm+lmls[j]*x^j
    return sm


#McKay-Thompson series w. a(0) = 24.
mt=[
[-1, 1],
[0, 24],
[1, 276],
[2, 2048],
[3, 11202],
[4, 49152],
[5, 184024],
[6, 614400],
[7, 1881471],
[8, 5373952],
[9, 14478180],
[10, 37122048],
[11, 91231550],
[12, 216072192],
[13, 495248952],
[14, 1102430208],
[15, 2390434947],
[16, 5061476352],
[17, 10487167336],
[18, 21301241856],
[19, 42481784514],
[20, 83300614144],
[21, 160791890304],
[22, 305854488576],
[23, 573872089212],
[24, 1063005978624],
[25,1945403602764],
[26, 3519965179904],
[27, 6300794030460],
[28, 11164248047616],
[29, 19591528119288],
[30, 34065932304384],
[31, 58718797964805],
[32, 100372723007488],
[33, 170215559855424],
[34, 286470013685760],
[35, 478625723149576],
[36, 794110053826560],
[37, 1308745319975256],
[38, 2143055278039040],
[39, 3487563372381816],
[40, 5641848336678912],
[41, 9074553043554568],
[42, 14515166263443456],
[43, 23093778743102262],
[44, 36552977852071936],
[45, 57567784186189368],
[46, 90226777113575424],
[47, 140752796480416011],
[48, 218578429975461888],
[49, 337945040343588276],
[50, 520271697765971968],
[51, 797652526220573580],
[52, 1218002527825723392],
[53, 1852604006634050072],
[54, 2807138079496716288],
[55, 4237760460302936433],
[56, 6374456847628238848],
[57, 9554873766107770560],
[58, 14273181657059143680],
[59, 21250450411204068910],
[60, 31535729115847852032],
[61, 46650835290143061624],
[62, 68797209365301886976],
[63, 101150679669913197462],
[64, 148280443106626633728],
[65, 216743142763626253712],
[66, 315923191441199824896],
[67, 459218611940943755226],
[68, 665710603285072019456],
[69, 962508846974918603904],
[70, 1388038765923851599872],
[71, 1996639069403279491427],
[72, 2864978197116521938944],
[73, 4100990608911708903432],
[74, 5856297079648098807808],
[75, 8343432715970391209502],
[76, 11859696700297921757184],
[77, 16820105145987654631552],
[78, 23802835313046730063872],
[79, 33611779636250175278886],
[80, 47362494062244172660736],
[81, 66600077798590855556532],
[82, 93460562353103053049856],
[83, 130891485964083426534122],
[84, 182952844329494181838848],
[85, 255227018229957765044016],
[86, 355376219286719031156736],
[87, 493899311443420857952845],
[88, 685157678128482627354624],
[89, 948763597225844233250504],
[90, 1311456320500974276980736],
[91, 1809633323386495729057992],
[92, 2492760414984152205361152],
[93, 3427959082742197097793024],
[94, 4706168520874397834575872],
[95, 6450411048962389429976770],
[96, 8826863296640622526464000],
[97, 12059665023346371597383976],
[98, 16450700321708764061517824],
[99, 22405985191605778318221966],
[100, 30470821082605141257437184]]

def mcKayThompson(n):return mt[n+1][1]

def sumOfRoots(f,prime,power):
    answer=0
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+a
    return answer

def productOfNonzeroRoots(f,prime,power):
    answer=1
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    #U.<x> = T[] -- this line gave an error
    poly = U(f)
    expr=str(poly)
    var('x')
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer*a
    return answer


def coercePolynomial(f,prime,power):
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    return poly

def constantTerm(f,prime,power):
    cp=coercePolynomial(f,prime,power)
    expr=str(cp)
    var('x')
    expr3=sage_eval(expr,locals={'x':0})
    return expr3


def roots(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    expr3=sage_eval(expr,locals={'x':0})
    if expr3==0:answer=answer+[0]
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            answer=answer+[a]
    return answer

def numericalTermOverFieldNoPrint(f,prime,power):
    T = GF(prime^power,'t')
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    xproduct=1
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        xproduct=xproduct*monomial^exponent
    if xproduct==0:return "error on numericalTermOverFieldNoPrint"
    return g/xproduct

def numericalTermOverFieldDigits(f,prime,power):
    ntf=numericalTermOverFieldNoPrint(f,prime,power)
    answer=ZZ(ntf)
    answer=answer.digits(prime)
    k=power
    answer=padListOnRightToLengthK(answer,k)
    return answer

def polynomialDegree(poly):
    return len(poly.list())-1

def leadingCoefficient(poly):
    dp=D(poly)
    deg=polynomialDegree(dp)
    return dp.list()[deg]

def testRootLogTimesPrimeCn(f,prime,power):
    tf=True
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            j=i*prime
            b=t^j
            expr4=sage_eval(expr,locals={'x':b})
            if expr4!=0:
                tf=False
                print (i,j)
                
    return tf

def basicRoots(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        if mod(i,prime)!=0:
            a=t^i
            expr3=sage_eval(expr,locals={'x':a})
            if expr3 == 0:answer=answer+[i]
    return answer

def basicRootsDigits(f,prime,power):
    answer=[]
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        if mod(i,prime)!=0:
            a=t^i
            expr3=sage_eval(expr,locals={'x':a})
            if expr3 == 0:answer=answer+[padListOnRightToLengthK(i.digits(prime),power)]
    return answer

def basicLogsAndMultiplicitiesOverField(f,prime,power):
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            if xm==0:data=data+[[-1,exponent]]
            if xm != 0:
                for i in [1..prime^power-1]:
                    if mod(i,prime)!=0:
                        if b^i==xm:data=data+[[i,exponent]]
    return data.sort()

def totalMultiplicityOverField(f,prime,power):
    tm=0
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:tm=tm+exponent
    return tm

def testRootLogTimesPrimeCn(f,prime,power):
    tf=True
    T = GF(prime^power)
    G.<t>= GF(prime^power)
    U.<x> = T[]
    poly = U(f)
    expr=str(poly)
    var('x')
    fp=poly.factor()
    lnf = len(fp) - 1
    flag = 1
    for i in [1..prime^power-1]:
        a=t^i
        expr3=sage_eval(expr,locals={'x':a})
        if expr3 == 0:
            j=i*prime
            b=t^j
            expr4=sage_eval(expr,locals={'x':b})
            if expr4!=0:
                tf=False
                print (i,j)
                
    return tf


def squeezePolynomial(f,prime,power):
    dg=f.degree()
    answer=0
    for k in [0..dg]:
        j=mod(k,prime^power-1)
        cf=polynomialCoefficient(k,f)
        cf=ZZ(mod(cf,prime))
        term=ZR(cf*x^j)
        answer=answer+term
    return answer

def completeSqueezePolynomial(f,prime,power):
    sp=squeezePolynomial(f,prime,power)
    dg=sp.degree()
    answer=0
    for k in [0..dg]:
        cf=polynomialCoefficient(k,sp)
        cf=ZZ(mod(cf,prime))
        term=cf*x^k
        answer=answer+term
    print answer
    return answer

def completeSqueezePolynomialNoPrint(f,prime,power):
    sp=squeezePolynomial(f,prime,power)
    dg=sp.degree()
    answer=0
    for k in [0..dg]:
        cf=polynomialCoefficient(k,sp)
        cf=ZZ(mod(cf,prime))
        term=cf*x^k
        answer=answer+term
    return answer



def frobenius(prime,x):return x^prime

def testDecreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]>list[k-1]:tf=False
    return tf

def testIncreasing(list):
    tf=True
    ln=len(list)
    for k in [1..ln-1]:
        if list[k]<list[k-1]:tf=False
    return tf

def is_composite(n):
    answer=False
    if (n>1)&(is_prime(n)==False):answer=True
    return answer

def reversalOfList(lst):
    return list(reversed(lst))

def differences(list):
    ln=len(list)
    answer=[]
    for k in [1..ln-1]:
        difference=list[k]-list[k-1]
        answer=answer+[difference]
    return answer

def zerosAndMultiplicitiesOverFieldWithPrint(f,prime,power): # n.b. roots, not root logs
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    print gf
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            data=data+[[xm,exponent]]
    return data

def zerosAndMultiplicitiesOverFieldWithOutPrint(f,prime,power): # n.b. roots, not root logs
    T = GF(prime^power,'t')
    b=T.multiplicative_generator()
    U.<x> = T[]
    g=U(f)
    gf=g.factor()
    lnf = len(gf)-1
    data=[]
    for nn in [0..lnf]:
        part = gf[nn]
        (monomial,exponent) = part
        mdx = monomial.degree()
        if mdx == 1:
            xm=x-monomial
            data=data+[[xm,exponent]]
    return data

In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snippeds
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [2]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [3]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=3
pairs=[]
for k in [0..200]:
    n=s[k][0]
    poly=D(s[k][1])
    polynum=ZR(polynomialnumerator(D(poly)))
    sd=leastSplittingDegree(polynum,prime,bound)
    pair=(mod(n,prime),sd)
    pairs=pairs+[pair]
print sort(union(pairs))

[(0, 1), (1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (2, 4)]


In [15]:
x=6;x.digits(4)

[2, 1]

In [2]:
import pickle
rfile =open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=120
prime=5
residue=2
splitting_degree=4
roots=[]
rootsP=[]
ratios=[]
nt_s=[]
ntP_s=[]
for k in [1..200]:
    n=s[k][0]
    if (mod(n,prime)==residue):
        poly=D(s[k][1])
        

        polynum=ZR(polynomialnumerator(D(poly)))
        sd=leastSplittingDegree(polynum,prime,bound)
        
        if sd==splitting_degree:
            size=prime^sd
            T = GF(size,'t')
            t=T.multiplicative_generator()
            U.<x> = T[]
            g=U(polynum)
            fg=g.factor()
            nt=numericalTermOverFieldNoPrint(g,prime,sd)
            nt_s=nt_s+[nt]
            ntdigits=numericalTermOverFieldDigits(g,prime,sd)
            lognt=ZZ(discreteLogOverField(nt,prime,sd))
            logntdigits=lognt.digits(prime)
            logntdigits=padListOnRightToLengthK(logntdigits,sd)
            lm=logsAndMultiplicitiesOverFieldDigits(polynum,prime,sd)
            roots=roots+[[n,nt,lm]]
            print "----------------------------------------------------------------------------------------------"
            print "n: ",n," splitting degree: ",sd," base ",prime,"nt: ",nt,"log nt:",lognt,
            print "nt digits:",ntdigits,"(log nt) digits:",logntdigits
            print "factored model:";print fg
            print "[root-log digits ,multiplicity] for each root:";print lm;print
            
        
        polynumP=D(polynomialnumeratorP(prime,D(poly)))
        sdP=leastSplittingDegree(polynumP,prime,bound)
        
        if sdP==splitting_degree:
            sizeP=prime^sdP
            TP = GF(sizeP,'t')
            tP=TP.multiplicative_generator()
            U.<x> = TP[]
            gP=U(polynumP)
            fgP=gP.factor()
            ntP=numericalTermOverFieldNoPrint(gP,prime,sdP)
            ntP_s=ntP_s+[ntP]
            ntdigitsP=numericalTermOverFieldDigits(gP,prime,sdP)
            logntP=ZZ(discreteLogOverField(ntP,prime,sdP))
            logntdigitsP=logntP.digits(prime)
            logntdigitsP=padListOnRightToLengthK(logntdigitsP,sdP)
            lmP=logsAndMultiplicitiesOverFieldDigits(polynumP,prime,sdP)
            rootsP=rootsP+[[n,ntP,lmP]]
            ratio=(g/gP).factor()
            ratios=ratios+[ratio]
            
            print "n: ",n," splitting degreeP: ",sdP," base ",prime,"ntP: ",ntP,"log ntP:",logntP,
            print "ntP digits:",ntdigitsP,"(log nt) digits:",logntdigitsP
            print "factored model:";print fgP
            print "[root-log digits ,multiplicity] for each root:";print lmP;print
            
            print "ratio:";print ratio
print "&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&"           
print "ratios:";print ratios;print
print "nt's:";print nt_s;print
print "ntP's:";print ntP_s 

wfile=open('/Users/barrybrent/run18april22p5res2sd4P','w')
u=pickle.dumps(str(rootsP))
wfile.write(u)
wfile.close()
# see corresponding 22 march file for the `non-P' data file

----------------------------------------------------------------------------------------------
n:  2  splitting degree:  4  base  5 nt:  2 log nt: 156 nt digits: [2, 0, 0, 0] (log nt) digits: [1, 1, 1, 1]
factored model:
(2) * (x + 2) * (x + 3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4)
[root-log digits ,multiplicity] for each root:
[[[3, 3, 3, 3], 1], [[1, 1, 1, 1], 1], [[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1]]

n:  2  splitting degreeP:  4  base  5 ntP:  2 log ntP: 156 ntP digits: [2, 0, 0, 0] (log nt) digits: [1, 1, 1, 1]
factored model:
(2) * (x + 2) * (x + 3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4)
[root-log digits ,multiplicity] for each root:
[[[3, 3, 3, 3], 1], [[1, 1, 1, 1], 1], [[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1]]

ratio:
1
------------------------------------------------------------------------------

n:  42  splitting degree:  4  base  5 nt:  2 log nt: 156 nt digits: [2, 0, 0, 0] (log nt) digits: [1, 1, 1, 1]
factored model:
(2) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^16 * (x + 4)^16 * (x + 2)^17 * (x + 3)^17
[root-log digits ,multiplicity] for each root:
[[[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1], [[2, 2, 2, 2], 16], [[4, 4, 4, 4], 16], [[3, 3, 3, 3], 17], [[1, 1, 1, 1], 17]]

n:  42  splitting degreeP:  4  base  5 ntP:  3 log ntP: 468 ntP digits: [3, 0, 0, 0] (log nt) digits: [3, 3, 3, 3]
factored model:
(3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^16 * (x + 4)^16 * (x + 2)^17 * (x + 3)^17
[root-log digits ,multiplicity] for each root:
[[[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1], [[2, 2, 2, 2], 16], [[4, 4, 4, 4], 16], [[3, 3, 3, 3], 17], [[1, 1, 1, 1], 17]]

ratio:
4
-----------------------

(3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^34 * (x + 4)^34 * (x + 2)^35 * (x + 3)^35
[root-log digits ,multiplicity] for each root:
[[[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1], [[2, 2, 2, 2], 34], [[4, 4, 4, 4], 34], [[3, 3, 3, 3], 35], [[1, 1, 1, 1], 35]]

n:  87  splitting degreeP:  4  base  5 ntP:  1 log ntP: 0 ntP digits: [1, 0, 0, 0] (log nt) digits: [0, 0, 0, 0]
factored model:
(x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^34 * (x + 4)^34 * (x + 2)^35 * (x + 3)^35
[root-log digits ,multiplicity] for each root:
[[[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1], [[2, 2, 2, 2], 34], [[4, 4, 4, 4], 34], [[3, 3, 3, 3], 35], [[1, 1, 1, 1], 35]]

ratio:
3
----------------------------------------------------------------------------------------------
n:  92  splitting degree:  4  base  5 nt:  1 log nt: 0 nt digit

----------------------------------------------------------------------------------------------
n:  127  splitting degree:  4  base  5 nt:  4 log nt: 312 nt digits: [4, 0, 0, 0] (log nt) digits: [2, 2, 2, 2]
factored model:
(4) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^50 * (x + 4)^50 * (x + 2)^51 * (x + 3)^51
[root-log digits ,multiplicity] for each root:
[[[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1], [[2, 2, 2, 2], 50], [[4, 4, 4, 4], 50], [[3, 3, 3, 3], 51], [[1, 1, 1, 1], 51]]

n:  127  splitting degreeP:  4  base  5 ntP:  3 log ntP: 468 ntP digits: [3, 0, 0, 0] (log nt) digits: [3, 3, 3, 3]
factored model:
(3) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^50 * (x + 4)^50 * (x + 2)^51 * (x + 3)^51
[root-log digits ,multiplicity] for each root:
[[[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1], [[2, 2, 2, 2], 5

n:  167  splitting degreeP:  4  base  5 ntP:  2 log ntP: 156 ntP digits: [2, 0, 0, 0] (log nt) digits: [1, 1, 1, 1]
factored model:
(2) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^66 * (x + 4)^66 * (x + 2)^67 * (x + 3)^67
[root-log digits ,multiplicity] for each root:
[[[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1], [[2, 2, 2, 2], 66], [[4, 4, 4, 4], 66], [[3, 3, 3, 3], 67], [[1, 1, 1, 1], 67]]

ratio:
2
----------------------------------------------------------------------------------------------
n:  172  splitting degree:  4  base  5 nt:  4 log nt: 312 nt digits: [4, 0, 0, 0] (log nt) digits: [2, 2, 2, 2]
factored model:
(4) * (x + t^2 + 2*t + 2) * (x + 4*t^2 + 3*t + 3) * (x + t^3 + 4*t^2 + 1) * (x + 4*t^3 + t^2 + 4) * (x + 1)^68 * (x + 4)^68 * (x + 2)^69 * (x + 3)^69
[root-log digits ,multiplicity] for each root:
[[[4, 3, 1, 1], 1], [[1, 1, 4, 3], 1], [[1, 4, 3, 1], 1], [[3, 1, 1, 4], 1], [[2, 2,